In [1]:
import pandas as pd
import numpy as np

In [2]:
# read in the IJH_2010.parquet.textClipping file
df = pd.read_parquet('IJH_2010.parquet')
df.head()

prcld   eps  prcod     cshtrd      cusip  \
date                      id                                                    
2010-06-30 00:00:00+00:00 064356_01  12.60 -2.10  13.01  4182555.0  171779309   
                          063080_01  40.49  1.44  41.21   379908.0  03662Q105   
                          118267_01  13.83  0.12  14.09   276324.0  500643200   
                          010540_01  34.60  2.11  35.33   436688.0  884315102   
                          024409_01  64.50  2.15  64.51  2398964.0  871130100   

                                     gsector  prccd        cik   tic  prchd  \
date                      id                                                  
2010-06-30 00:00:00+00:00 064356_01     45.0  12.68   936395.0  CIEN  13.18   
                          063080_01     45.0  40.57  1013462.0  ANSS  41.57   
                          118267_01     20.0  13.90    56679.0   KFY  14.30   
                          010540_01     20.0  34.70    97854.0   TNB  35.69   
                          024409_01     45.0  64.66   768262.0  SY.3  64.73   

                                          cshoc  
date                      id                     
2010-06-30 00:00:00+00:00 064356_01  93094000.0  
                          063080_01  90509000.0  
                          118267_01  45449000.0  
                          010540_01  52795000.0  
                          024409_01  86861000.0

In [3]:
new_index = []
for tup in df.index.values:
    new_index.append((tup[0], df.loc[tup]['tic']))

In [38]:
new_idx = pd.MultiIndex.from_tuples(new_index, names=['date','ticker'])

In [40]:
# Making a new dataframe where the column are the ticker names and the rows are the close prices for each day.
df.set_index(new_idx, inplace=True)
#df.drop(columns=['id'], axis=1, inplace=True)
df.index


MultiIndex([('2010-06-30 00:00:00+00:00',  'CIEN'),
            ('2010-06-30 00:00:00+00:00',  'ANSS'),
            ('2010-06-30 00:00:00+00:00',   'KFY'),
            ('2010-06-30 00:00:00+00:00',   'TNB'),
            ('2010-06-30 00:00:00+00:00',  'SY.3'),
            ('2010-06-30 00:00:00+00:00', 'PCXCQ'),
            ('2010-06-30 00:00:00+00:00', 'NSR.3'),
            ('2010-06-30 00:00:00+00:00',   'JLL'),
            ('2010-06-30 00:00:00+00:00',  'QSFT'),
            ('2010-06-30 00:00:00+00:00',   'GGG'),
            ...
            ('2022-11-01 00:00:00+00:00',   'SWN'),
            ('2022-11-01 00:00:00+00:00',   'ONB'),
            ('2022-11-01 00:00:00+00:00',   'NFG'),
            ('2022-11-01 00:00:00+00:00',  'LECO'),
            ('2022-11-01 00:00:00+00:00',  'STAA'),
            ('2022-11-01 00:00:00+00:00',     'M'),
            ('2022-11-01 00:00:00+00:00',  'NAVI'),
            ('2022-11-01 00:00:00+00:00',   'AFG'),
            ('2022-11-01 00:00:00+00:00',  'FFIN

In [43]:
df = df.drop(columns=['tic'], axis=1)
df.drop(columns=['prcld', 'eps','prcod', 'cshtrd', 'cusip', 'gsector', 'cik', 'prchd', 'cshoc'], axis=1, inplace=True)

In [97]:
from collections import OrderedDict

unstacked_close = df.stack().unstack(level=1) # stack and unstack at second level of multiindex as an inner transpose
date_index = list(OrderedDict.fromkeys([tup[0] for tup in df.index.values])) # convert multiindex to single index

AttributeError: 'Timestamp' object has no attribute 'astype'

In [99]:
unstacked_close.set_index(pd.Index(date_index, name='date').astype(int), inplace=True) # set new index as integer date

In [101]:
unstacked_close

ticker,AA,AAI,AAN.2,AAP,ABMD,ACC,ACF,ACHC,ACIIQ,ACIW,...,X,XEC,XLS,XPO,XRX,Y,YELP,YETI,ZBRA,ZD
date,,,,,,,,,,,,,,,,,,,,,
1277856000000000000,NaN,4.85,17.07,50.18,NaN,NaN,18.22,NaN,198.1,6.490000,...,NaN,71.58,NaN,NaN,NaN,NaN,NaN,NaN,25.37,NaN
1277942400000000000,NaN,4.85,16.78,51.70,NaN,NaN,18.66,NaN,197.9,6.663333,...,NaN,70.14,NaN,NaN,NaN,NaN,NaN,NaN,24.77,NaN
1278028800000000000,NaN,4.76,16.44,51.01,NaN,NaN,18.45,NaN,193.2,6.476667,...,NaN,70.17,NaN,NaN,NaN,NaN,NaN,NaN,24.66,NaN
1278374400000000000,NaN,4.64,16.44,50.50,NaN,NaN,18.37,NaN,197.4,6.386667,...,NaN,70.88,NaN,NaN,NaN,NaN,NaN,NaN,24.37,NaN
1278460800000000000,NaN,4.84,17.38,50.89,NaN,NaN,19.50,NaN,207.8,6.500000,...,NaN,74.76,NaN,NaN,NaN,NaN,NaN,NaN,25.21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666742400000000000,42.65,NaN,NaN,NaN,NaN,NaN,NaN,81.65,NaN,23.530000,...,21.95,NaN,NaN,50.35,14.48,NaN,NaN,32.58,NaN,77.65
1666828800000000000,41.41,NaN,NaN,NaN,NaN,NaN,NaN,82.31,NaN,23.860000,...,21.41,NaN,NaN,49.42,14.67,NaN,NaN,31.92,NaN,76.38
1666915200000000000,39.49,NaN,NaN,NaN,NaN,NaN,NaN,83.05,NaN,24.740000,...,20.51,NaN,NaN,50.67,14.40,NaN,NaN,32.70,NaN,78.59


Beginning of backtest

In [76]:
# calculate the daily returns
daily_returns = df.pct_change()  # calculate the daily returns
mkt_daily_returns = daily_returns.mean(axis=1)  # calculate the market return, which is also mean of the daily returns

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [77]:
# bring the spx data into the notebook as a dataframe
df = pd.read_table('SPX_20071123.txt')  # stores day closes for SPX components
df['Date'] = df['Date'].astype('int')  # convert the date to an integer
df.set_index('Date', inplace=True)  # set the index to the date
df.sort_index(inplace=True)  # sort the dataframe by date
df.head(10)

,A,AA,AAPL,ABC,ABI,ABK,ABT,ACAS,ACE,ACS,...,XEL,XL,XLNX,XOM,XRX,XTO,YHOO,YUM,ZION,ZMH
Date,,,,,,,,,,,,,,,,,,,,,
19991124,41.1250,20.3772,23.6563,5.9980,NaN,31.1549,22.8971,7.6786,12.8492,19.0000,...,14.8614,38.5249,43.8110,35.1539,26.7738,2.2456,57.7500,10.0716,51.6312,NaN
19991126,41.1875,20.6142,23.7656,5.9980,NaN,31.1183,22.5998,7.6320,12.8492,18.7188,...,14.9079,38.5249,43.5356,34.6541,26.7738,2.2215,56.7188,9.8871,51.6826,NaN
19991129,42.0000,20.5352,23.6406,5.9675,NaN,30.6418,23.0086,7.5855,12.2246,18.7188,...,14.8614,38.2478,46.8092,35.2650,26.2989,2.0403,56.5313,9.9640,50.7074,NaN
19991130,42.1875,20.6932,24.4688,6.0284,NaN,31.9613,22.5998,7.4925,12.1353,18.7188,...,15.0926,37.6935,43.8110,35.2372,25.7052,1.9679,53.1875,10.2100,53.0170,NaN
19991201,43.0000,21.1078,25.7656,6.0588,NaN,33.5740,22.2652,7.5390,12.2692,18.7188,...,15.0460,37.7859,43.7804,36.3201,25.3584,1.9317,57.2188,10.0101,53.6329,NaN
19991202,44.1250,22.1346,27.5469,5.9066,NaN,32.5477,22.1537,7.7019,12.4030,18.5625,...,15.0460,37.2315,46.3197,36.3201,25.4182,1.9317,61.4531,9.9179,54.0947,NaN
19991203,45.0000,22.4703,28.7500,5.7239,NaN,33.3541,22.2652,7.6786,12.6707,18.8750,...,15.1849,37.8783,43.1991,36.7089,24.8201,1.9075,63.2500,9.8410,55.4805,NaN
19991206,45.7500,22.6283,29.0938,5.8153,NaN,33.4640,21.6333,7.7717,12.3138,18.6563,...,15.2306,37.5087,44.5452,37.2643,24.9995,1.8713,70.2031,9.8410,54.0947,NaN
19991207,45.2500,23.0034,29.4531,5.7544,NaN,33.4274,21.2244,7.6320,12.1799,18.8438,...,15.1849,37.2315,43.4744,37.2643,24.4014,1.8955,87.0000,9.9025,52.2471,NaN
